1. txt파일을 읽는다.
2. xywh -> xyxy변환
3. 0~1 -> -1~+1 변환
4. warp 변환
5. -1 ~ +1 변환
6. xyxy -> xywh변환
7. txt 저장

In [34]:
from fastai.vision.augment import Warp
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
from fastai.vision.core import TensorImage ,TensorBBox , TensorPoint
import os
import shutil
import random
import tqdm

In [35]:
def xywh2xyxy(x):
    y = np.copy(x)
    y[..., 0] = x[..., 0] - x[..., 2] / 2  # top left x
    y[..., 1] = x[..., 1] - x[..., 3] / 2  # top left y
    y[..., 2] = x[..., 0] + x[..., 2] / 2  # bottom right x
    y[..., 3] = x[..., 1] + x[..., 3] / 2  # bottom right y
    return y

def xyxy2xywh(x):
    y = np.copy(x)
    y[..., 0] = (x[..., 0] + x[..., 2]) / 2  # x center
    y[..., 1] = (x[..., 1] + x[..., 3]) / 2  # y center
    y[..., 2] = x[..., 2] - x[..., 0]  # width
    y[..., 3] = x[..., 3] - x[..., 1]  # height
    return y

def zero2minusone(x):
    y = np.copy(x)
    y = 2 * y - 1
    return y

def minusone2zero(x):
    y = np.copy(x)
    y = (y + 1) / 2
    return y

In [36]:
from_image_path = './Final_C_data/train/images'
from_label_path = './Final_C_data/train/labels'

to_image_path = './Final_C_data/train/warping_images'
to_label_path = './Final_C_data/train/warping_labels'

image_list = os.listdir(from_image_path)

ratio = 0.25
image_list = random.sample(image_list, int(len(image_list)*ratio))
dx = [-0.4 , -0.3 , -0.2 , -0.1 , 0.1 , 0.2 , 0.3 , 0.4]
dy = [-0.4 , -0.3 , -0.2 , -0.1 , 0.1 , 0.2 , 0.3 , 0.4]


## for 문 돌리기.
for image_name in tqdm.tqdm(image_list):
    file_name = os.path.splitext(image_name)[0]
    img = cv2.imread(os.path.join(from_image_path,image_name))
    img = torch.tensor(img) / 255.0
    img = img.permute(2, 0, 1)

    label_xywh = [] # txt파일 class,x,y,w,h 저장
    with open(os.path.join(from_label_path,file_name+'.txt'), 'r',encoding="UTF-8") as file:
        for line in file:
            label_xywh.append(list(map(float,line.strip().split())))
    label_xywh = np.array(label_xywh)
    label_xyxy= zero2minusone(xywh2xyxy(label_xywh[:,1:]))
    label_xyxy = torch.Tensor(label_xyxy)


    xi = random.randint(0, 7)
    yi = random.randint(0, 7)
    warp_transform = Warp(p=1. , draw_y=dy[yi], draw_x=dx[yi] , size=(640,640))

    warped_image = warp_transform(TensorImage(img.unsqueeze(0)))[0]
    warped_label = warp_transform(TensorBBox(label_xyxy.unsqueeze(0)))[0]

    warped_image = np.transpose(warped_image.numpy(), (1, 2, 0)) * 255
    warped_label = xyxy2xywh(minusone2zero(warped_label.numpy()))
    warped_label = np.hstack((label_xywh[:,0].reshape(-1,1), warped_label))

    cv2.imwrite(os.path.join(to_image_path,file_name+'warped.jpg'),warped_image)
    with open(os.path.join(to_label_path,file_name+'warped.txt'), 'w') as file:
        for obj in warped_label:
            file.write(' '.join([str(int(obj[0]))] + list(map(str, obj[1:]))) + '\n')
    


100%|██████████| 9381/9381 [12:28<00:00, 12.54it/s]


In [38]:
#이미지 테스트
to_image_path = './Final_C_data/train/warping_images'
to_label_path = './Final_C_data/train/warping_labels'

save_path = './warp_test'

image_list = os.listdir(to_image_path)
image_list = random.sample(image_list, 20)
# 이미지 로드
for idx , image_name in enumerate(image_list):
    file_name = os.path.splitext(image_name)[0]
    image = cv2.imread(os.path.join(to_image_path,image_name))
   
    height, width, _ = image.shape
    with open(os.path.join(to_label_path,file_name + '.txt'), "r") as file:
        lines = file.readlines()  # 파일의 모든 줄을 읽어옴

        for line in lines:
            line = line.strip()  # 줄의 양쪽 공백 및 개행문자 제거
            values = line.split(" ")  # 쉼표와 공백을 기준으로 문자열을 나눔

            # 값 추출
            class_name = int(values[0])
            box_x = float(values[1])
            box_y = float(values[2])
            box_w = float(values[3])
            box_h = float(values[4])

            # 좌표 값으로 bounding box 그리기
            left_x = int((box_x-box_w/2)*width)
            left_y = int((box_y-box_h/2)*height)
            right_x = int((box_x+box_w/2)*width)
            right_y = int((box_y+box_h/2)*height)
            cv2.rectangle(image, (left_x ,left_y), (right_x, right_y), (0, 255, 0), 2)

    cv2.imwrite(os.path.join(save_path,f"{str(idx)}.png"), image)